<a href="https://colab.research.google.com/github/Rochak-Sharma/deepFakes/blob/main/deepFakeApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We'll manage our dependencies using Pipenv. Here's the complete Pipfile:

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 19 18:24:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install flask
!pip install flask-ngrok

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/deepfakes/first-order-model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/deepfakes/first-order-model


In [5]:
import moviepy.editor as mpe 
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from IPython.display import Image
from IPython.display import clear_output
import warnings
 
from demo import make_animation
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")
from demo import load_checkpoints
 
 
def display(source, driving, generated1=None, generated2 = None):
    fig = plt.figure(figsize=(8 + 4 * (generated1 is not None) + 4 * (generated2 is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated1 is not None:
            cols.append(generated1[i])
        if generated2 is not None:
            cols.append(generated2[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


path_ = "/content/drive/MyDrive/deepfakes/first-order-model/config/"
Checkpoint = 'vox-adv-cpk.pth'
checkpoint_p = {
    "vox-cpk.pth": path_ + 'vox-cpk.pth.tar', 
    "vox-adv-cpk.pth": path_ + 'vox-adv-cpk.pth.tar'
}
  
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=str(checkpoint_p[Checkpoint]))
 

print("CONFIGURE DONE")

CONFIGURE DONE


In [6]:
def combine_audio(vidname, audname, outname, fps=25):
    import moviepy.editor as mpe
    my_clip = mpe.VideoFileClip(vidname)
    audio_background = mpe.AudioFileClip(audname)
    final_clip = my_clip.set_audio(audio_background)
    final_clip.write_videofile(outname,fps=fps)



def Predictor(user_image_path, imposer_file_path, mp3):
  print("user_image_path", user_image_path, "imposer_file_path", imposer_file_path, "mp3", mp3);
  
  Source_Image = user_image_path
  Driving_Video = imposer_file_path

  source_image = imageio.imread(Source_Image)
  driving_video = imageio.mimread(str(Driving_Video))
  
  #Resize image and video to 256x256
  
  source_image = resize(source_image, (256, 256))[..., :3]
  driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
  
  #predictions1 = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
  #save resulting video
  #video can be downloaded from /content folder
  predictions2 = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
  #HTML(display(source_image, driving_video, predictions1, predictions2).to_html5_video())
  imageio.mimsave('/content/drive/MyDrive/deepfakes/generated_expressionP2.mp4', [img_as_ubyte(frame) for frame in predictions2])
  combine_audio('/content/drive/MyDrive/deepfakes/generated_expressionP2.mp4', mp3,'/content/drive/MyDrive/deepfakes/audio&video.mp4')

In [10]:
import os
from uuid import uuid4
from flask import *
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, send_from_directory, render_template, Response
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import time
from flask import send_file
import os
from flask import Flask, jsonify
from flask import Response
import cv2
import cv2
import numpy as np
from markupsafe import escape


In [11]:
app = Flask(__name__)

@app.route("/upload/<song>", methods=["POST"])
def upload(song):
    print(f"ID : {escape(song)}", type(escape(song)), song)
    pic = request.files['pic']
    print(pic.filename)
    pic.save('/content/drive/MyDrive/deepfakes/' + pic.filename)
    user_name = '/content/drive/MyDrive/deepfakes/' + pic.filename
    Predictor(user_name, '/content/drive/MyDrive/deepfakes/video&audioMain/{}.mp4'.format(song),'/content/drive/MyDrive/deepfakes/video&audioMain/{}.mp3'.format(song))
    
    return "Image upload done"

@app.route("/")
def index():
    return "/ slash page"

@app.route('/video_feed')
def video_feed():
    #Video streaming route. Put this in the src attribute of an img tag
    try:
      return send_file("/content/drive/MyDrive/deepfakes/audio&video.mp4", mimetype='multipart/x-mixed-replace; boundary=frame')
    except:
      print(">><><><><><   EXCEPT SATEMENT><><><>  ><<<><<")
    

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://23d0-35-247-49-75.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Sep/2021 18:32:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:32:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Sep/2021 18:32:27] "GET / HTTP/1.1" 200 -


ID : Song 1 <class 'markupsafe.Markup'> Song 1
image_cropper_1632076404561.jpg
user_image_path /content/drive/MyDrive/deepfakes/image_cropper_1632076404561.jpg imposer_file_path /content/drive/MyDrive/deepfakes/video&audioMain/Song 1.mp4 mp3 /content/drive/MyDrive/deepfakes/video&audioMain/Song 1.mp3


...

100%|██████████| 150/150 [00:00<00:00, 754.65it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/drive/MyDrive/deepfakes/audio&video.mp4



100%|█████████▉| 375/376 [00:00<00:00, 589.58it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/deepfakes/audio&video.mp4 



127.0.0.1 - - [19/Sep/2021 18:36:10] "POST /upload/Song%201 HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:36:11] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:36:12] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:36:13] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:36:18] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:36:18] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2021 18:36:19] "GET /video_feed HTTP/1.1" 200 -
